# NextStep
NextStep is a tool to extract time-series features of history stock prices and then based on these feature, predict future changes using machine learning models. 

## Method
Currently, features used in this tool include adjusted closing price, volume, weekday(Others reported that prices on Fridays is more likely to increase than those on Mondays), rate of change, momentum, williams %R, relative strength index, commodity channel index, simple moving average, exponential moving averages, moving average convergence divergence, average directional index, money flow index, time series forcasting, triple exponential moving average, on balance volumn, average true range, and Bollinger Bands.

Different feature selection methods were applied to extracted features respectively to pick the most important features, including recursive feature elimination(RFE) and random forest classifier(RF). PCA were also adopted to reduce the dimensions of our features. Six procedures were used in my test: no feature selection, PCA only, RF only, RF + PCA, RFE only, RFE + PCA.

Then, support vector machine(SVM) and neural network(NN) were trained with totally 1370 samples, after removing dates with NA features. For each set of features modified by different feature selection method, the best parameters for SVM and NN model were selected based on precision score(I prefer to lose less money than find more dates that will potentially make money. You can set a different scoring way in the tool).

Finally, all models were sorted based on their precision scores. And the top models will be used to predict test data sets and figure out their generalizarion capacity.

## Results
1. predict next day price
Apple(AAPL) history stock prices from 1/3/2012 to 12/29/2017 were split into training(1370 sample) and testing data(100 samples) sets. The cross validation precision scores of training data using different models were within a narrow range of 0.52-0.58. The plots below showed ROC curve of different models applied to testing data. Overall, neural network had a better performance than SVM.

In [1]:
from IPython.display import HTML, display
display(HTML("<table><tr><td><img src='./figures/top6_ROC.jpg'></td><td><img src='./figures/7th-12th_ROC.jpg'></td></tr></table>"))

,


From the ROC curve above, the best model in testing data was RF + NN (on test data: precision = 0.61, auc=0.61). And the learning curve of this model is shown bellow. The training score was very high, while cv score was much lower. And the cv score didn't change much as sample size grew. There was high variance in the model.

In [4]:
%%html
<img src='./figures/RF.NN_classifier_learning_curve.png'>

Therefore, I tried to set cutting threshold of RF selectoin from 25% to 10%. The number of feature increased from 51 to 62. But, there was no improvement. (on test data: precision = 0.51, auc=0.51)

In [5]:
%%html
<img src='./figures/RF10.NN_classifier_learning_curve.png'>

So I made more polynomial features (degree=2) and then selected important ones using RF, percentile=25). Still no improvement.(on test data: precision = 0.57, auc=0.56)

In [6]:
%%html
<img src='./figures/polyRF.NN_classifier_learning_curve.png'>

2. predict the price 5 days later
As next day proce change prediction result was not satisfying. I tried to predict change of stock price 5 days later. This time, cross validation precision scores of prediction of stock price change 5 days later were within a range of 0.74 to 0.78. And precision score of test data were within 0.81-0.91. These models were precise and with good generalizability. And from the learning curve of RF + PCA, NN model, if more samples were used to trained this model, better result can be achieved.

In [8]:
display(HTML("<table><tr><td><img src='./figures/5d/top6_ROC.jpg'></td><td><img src='./figures/5d/RF+PCA.NN_classifier_learning_curve.png'></td></tr></table>"))

,


From the top 6 models, RF worked better as a feature selection method than RFE. And the important features selected by RF to predict 5 days APPL price were 18 SP&500 index features, 16 NASDAQ index features and 16 APPL features. So outside has a great influence on APPL price changes 5 days later. Among these features, many of them represent long-term moving characteristics of stock price, like EMA and MACD. Volatility signal also played a role, such as Bollinger Bands®. 

3. predict the price 10, 15, 20 days later
RF + NN models also did good jobs in predicting AAPL changes in 10, 15 and 20 days.

In [10]:
display(HTML("<table><tr><td><img src='./figures/10d/top6_ROC.jpg'></td><td><img src='./figures/15d/top6_ROC.jpg'></td><td><img src='./figures/20d/top6_ROC.jpg'></td></tr></table>"))

,,


## Conclusion
NextStep as stock price prediction tool is precise and generalizable in predicting changes in 5 days or longer. Among the feature selection methods and machine learning models considered in this tool, feature selection based on random forest classifier feature importance and neural netowrk classifier had better performance. As for APPL, long-term moving features and volatility signal were essential in prediction, as well as the states of the whole market.